In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

In [20]:
dataset = ''
with open('Merchant of Venice.txt', 'r') as f:
    # Read the file line by line
    for line in f:
        # Print or process each line
        if line[0] == ' ':
            line = line.lower()
            line = line.strip()
            line = ''.join([char for char in line if char not in [',', '.', "'", '[', ']', '!', '-', ':', ';', '?']])
            dataset += line

In [21]:
print(dataset)

in sooth i know not why i am so sadit wearies me you say it wearies youbut how i caught it found it or came by itwhat stuff tis made of whereof it is borni am to learnand such a wantwit sadness makes of methat i have much ado to know myselfyour mind is tossing on the oceanthere where your argosies with portly saillike signiors and rich burghers on the floodor as it were the pageants of the seado overpeer the petty traffickersthat curtsy to them do them reverenceas they fly by them with their woven wingsbelieve me sir had i such venture forththe better part of my affections wouldbe with my hopes abroad i should be stillplucking the grass to know where sits the windpeering in maps for ports and piers and roadsand every object that might make me fearmisfortune to my ventures out of doubtwould make me sadmy wind cooling my brothwould blow me to an ague when i thoughtwhat harm a wind too great at sea might doi should not see the sandy hourglass runbut i should think of shallows and of flats

In [22]:
chars = sorted(list(set(''.join(dataset))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
print(itos)
print(stoi)

{0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [23]:
class NextChar(nn.Module):
  def __init__(self, context, vocab_size, emb_dim, hidden_size):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, emb_dim)
    self.lin1 = nn.Linear(context * emb_dim, hidden_size)
    self.lin2 = nn.Linear(hidden_size, vocab_size)

  def forward(self, x):
    x = self.emb(x)
    x = x.view(x.shape[0], -1)
    x = torch.sin(self.lin1(x))
    x = self.lin2(x)
    return x

In [24]:
context = 10
emb_size = 30
hidden_layer_size = 10

In [25]:
for i in [30]:
    print("emb_size", i)
    emb_size = i
    for j in [10]:
        print("\thidden_layer_size", j)
        hidden_layer_size = j
        X, Y = [], []
        for i in range(context, len(dataset)):
            x = [stoi[dataset[i-k]] for k in range(context, 0, -1)]
            X.append(x)
            Y.append(stoi[dataset[i]])
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        model = NextChar(context, len(stoi), emb_size, hidden_layer_size)
        loss_fn = nn.CrossEntropyLoss()
        opt = torch.optim.AdamW(model.parameters(), lr=0.01)
        # Mini-batch training
        batch_size = 4096
        elapsed_time = []
        for epoch in range(1000):
            for i in range(0, X.shape[0], batch_size):
                x = X[i:i+batch_size]
                y = Y[i:i+batch_size]
                y_pred = model(x)
                loss = loss_fn(y_pred, y)
                loss.backward()
                opt.step()
                opt.zero_grad()
        torch.save(model.state_dict(), 'trained_models/{}_{}_{}'.format(context, emb_size, hidden_layer_size))

emb_size 30
	hidden_layer_size 10


In [26]:
def get_k_chars(input, k, model, c):
    result = input
    context = input[-c:]
    for count in range(k):
        x = [[stoi[char] for char in context]]
        x = torch.tensor(x)
        y_pred = model(x)
        y = itos[torch.distributions.categorical.Categorical(logits=y_pred).sample().item()]
        result+=y
        context = context[1:] + y
    return result

In [27]:
model.load_state_dict(torch.load('trained_models/{}_{}_{}'.format(context, emb_size, hidden_layer_size)))
text_input = 'Your mind is tossing o'
k = 20
result = get_k_chars(text_input, k, model, context)

In [28]:
result

'Your mind is tossing of trecurtyourserii o'